In [1]:
import pandas as pd
import numpy as np

In [2]:
# Reading in the data
shoe_data = pd.read_csv('/Users/logno/Documents/Home/BAF1/ds_shoe_proj/data/Clean_Shoe_Data.csv', parse_dates = True)
df = shoe_data.copy()
df

Order Date   Brand                                   Sneaker Name  \
0      2017-09-01   Yeezy           Adidas Yeezy Boost 350 Low V2 Beluga   
1      2017-09-01   Yeezy    Adidas Yeezy Boost 350 V2 Core Black Copper   
2      2017-09-01   Yeezy     Adidas Yeezy Boost 350 V2 Core Black Green   
3      2017-09-01   Yeezy       Adidas Yeezy Boost 350 V2 Core Black Red   
4      2017-09-01   Yeezy  Adidas Yeezy Boost 350 V2 Core Black Red 2017   
...           ...     ...                                            ...   
99951  2019-02-13   Yeezy    adidas Yeezy Boost 350 V2 Static Reflective   
99952  2019-02-13   Yeezy    adidas Yeezy Boost 350 V2 Static Reflective   
99953  2019-02-13   Yeezy    adidas Yeezy Boost 350 V2 Static Reflective   
99954  2019-02-13   Yeezy    adidas Yeezy Boost 350 V2 Static Reflective   
99955  2019-02-13   Yeezy    adidas Yeezy Boost 350 V2 Static Reflective   

       Sale Price  Retail Price Release Date  Shoe Size  Buyer Region  
0            1097           220   2016-09-24       11.0    California  
1             685           220   2016-11-23       11.0    California  
2             690           220   2016-11-23       11.0    California  
3            1075           220   2016-11-23       11.5      Kentucky  
4             828           220   2017-02-11       11.0  Rhode Island  
...           ...           ...          ...        ...           ...  
99951         565           220   2018-12-26        8.0        Oregon  
99952         598           220   2018-12-26        8.5    California  
99953         605           220   2018-12-26        5.5      New York  
99954         650           220   2018-12-26       11.0    California  
99955         640           220   2018-12-26       11.5         Texas  

[99956 rows x 8 columns]

# Data Cleaning

In [3]:
# Renaming columns to get rid of spaces 
df = df.rename(columns={
    "Order Date": "Order_date",
    "Sneaker Name": "Sneaker_Name",
    "Sale Price": "Sale_Price",
    "Retail Price": "Retail_Price",
    "Release Date": "Release_Date",
    "Shoe Size": "Shoe_Size",
    "Buyer Region": "Buyer_Region"
    })

In [4]:
# Converting dates into numericals
import datetime as dt

df['Order_date'] = pd.to_datetime(df['Order_date'])
df['Order_date']=df['Order_date'].map(dt.datetime.toordinal)

df['Release_Date'] = pd.to_datetime(df['Release_Date'])
df['Release_Date']=df['Release_Date'].map(dt.datetime.toordinal)

In [5]:
# Setting up train & test data
from sklearn.model_selection import train_test_split

X = df.drop(['Sale_Price'], axis=1)
y = df.Sale_Price
X_train,X_valid,y_train,y_valid = train_test_split(X, y, test_size=0.2, random_state = 27)

In [6]:
# Converting categorical data to numerical
from sklearn.preprocessing import OneHotEncoder

object_cols = ['Sneaker_Name', 'Buyer_Region', 'Brand']
# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# # Adding the column names after one hot encoding
OH_cols_train.columns = OH_encoder.get_feature_names(object_cols)
OH_cols_valid.columns = OH_encoder.get_feature_names(object_cols)

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

In [7]:
X_train.head()

Order_date  Retail_Price  Release_Date  Shoe_Size  \
5827       736663           220        736658       11.0   
168        736581           220        736385        8.0   
80966      737055           170        737047        8.0   
88639      737070           170        737047       10.0   
3292       736657           160        736581       10.0   

       Sneaker_Name_Adidas Yeezy Boost 350 Low Moonrock  \
5827                                                0.0   
168                                                 0.0   
80966                                               0.0   
88639                                               0.0   
3292                                                0.0   

       Sneaker_Name_Adidas Yeezy Boost 350 Low Oxford Tan  \
5827                                                 0.0    
168                                                  0.0    
80966                                                0.0    
88639                                                0.0    
3292                                                 0.0    

       Sneaker_Name_Adidas Yeezy Boost 350 Low Pirate Black 2015  \
5827                                                 0.0           
168                                                  0.0           
80966                                                0.0           
88639                                                0.0           
3292                                                 0.0           

       Sneaker_Name_Adidas Yeezy Boost 350 Low Pirate Black 2016  \
5827                                                 0.0           
168                                                  0.0           
80966                                                0.0           
88639                                                0.0           
3292                                                 0.0           

       Sneaker_Name_Adidas Yeezy Boost 350 Low Turtledove  \
5827                                                 0.0    
168                                                  0.0    
80966                                                0.0    
88639                                                0.0    
3292                                                 0.0    

       Sneaker_Name_Adidas Yeezy Boost 350 Low V2 Beluga  ...  \
5827                                                 0.0  ...   
168                                                  0.0  ...   
80966                                                0.0  ...   
88639                                                0.0  ...   
3292                                                 0.0  ...   

       Buyer_Region_Texas  Buyer_Region_Utah  Buyer_Region_Vermont  \
5827                  0.0                0.0                   0.0   
168                   0.0                0.0                   0.0   
80966                 0.0                0.0                   0.0   
88639                 0.0                0.0                   0.0   
3292                  0.0                0.0                   0.0   

       Buyer_Region_Virginia  Buyer_Region_Washington  \
5827                     0.0                      0.0   
168                      0.0                      0.0   
80966                    0.0                      0.0   
88639                    0.0                      0.0   
3292                     0.0                      0.0   

       Buyer_Region_West Virginia  Buyer_Region_Wisconsin  \
5827                          0.0                     0.0   
168                           0.0                     0.0   
80966                         0.0                     0.0   
88639                         0.0                     0.0   
3292                          0.0                     0.0   

       Buyer_Region_Wyoming  Brand_ Yeezy  Brand_Off-White  
5827                    0.0           1.0              0.0  
168                     0.0           1.0              0.0  
80966                   0.0       

In [8]:
X_train.shape

(79964, 107)

In [9]:
X_valid.head()

Order_date  Retail_Price  Release_Date  Shoe_Size  \
30843      736865           190        736868        6.0   
74873      737043           220        736651        9.0   
99408      737102           160        737097        8.5   
89424      737072           220        737055        9.0   
76890      737047           220        737021        7.5   

       Sneaker_Name_Adidas Yeezy Boost 350 Low Moonrock  \
30843                                               0.0   
74873                                               0.0   
99408                                               0.0   
89424                                               0.0   
76890                                               0.0   

       Sneaker_Name_Adidas Yeezy Boost 350 Low Oxford Tan  \
30843                                                0.0    
74873                                                0.0    
99408                                                0.0    
89424                                                0.0    
76890                                                0.0    

       Sneaker_Name_Adidas Yeezy Boost 350 Low Pirate Black 2015  \
30843                                                0.0           
74873                                                0.0           
99408                                                0.0           
89424                                                0.0           
76890                                                0.0           

       Sneaker_Name_Adidas Yeezy Boost 350 Low Pirate Black 2016  \
30843                                                0.0           
74873                                                0.0           
99408                                                0.0           
89424                                                0.0           
76890                                                0.0           

       Sneaker_Name_Adidas Yeezy Boost 350 Low Turtledove  \
30843                                                0.0    
74873                                                0.0    
99408                                                0.0    
89424                                                0.0    
76890                                                0.0    

       Sneaker_Name_Adidas Yeezy Boost 350 Low V2 Beluga  ...  \
30843                                                0.0  ...   
74873                                                0.0  ...   
99408                                                0.0  ...   
89424                                                0.0  ...   
76890                                                0.0  ...   

       Buyer_Region_Texas  Buyer_Region_Utah  Buyer_Region_Vermont  \
30843                 0.0                0.0                   0.0   
74873                 1.0                0.0                   0.0   
99408                 0.0                0.0                   0.0   
89424                 0.0                0.0                   0.0   
76890                 0.0                0.0                   0.0   

       Buyer_Region_Virginia  Buyer_Region_Washington  \
30843                    0.0                      0.0   
74873                    0.0                      0.0   
99408                    0.0                      0.0   
89424                    0.0                      0.0   
76890                    0.0                      0.0   

       Buyer_Region_West Virginia  Buyer_Region_Wisconsin  \
30843                         0.0                     0.0   
74873                         0.0                     0.0   
99408                         0.0                     0.0   
89424                         0.0                     0.0   
76890                         0.0                     0.0   

       Buyer_Region_Wyoming  Brand_ Yeezy  Brand_Off-White  
30843                   0.0           0.0              1.0  
74873                   0.0           1.0              0.0  
99408                   0.0       

# Pipelines Creation
## 1. Data Preprocessing by using Standard Scaler
## 2. Reduce Dimension using PCA
## 3. Apply  Classifier

In [10]:
# Importing models
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.tree import DecisionTreeRegressor

In [11]:
# Setting up pipelines

from sklearn.pipeline import Pipeline
# Decision Tree Regression Pipeline
pipeline_dtr=Pipeline([('dtr', DecisionTreeRegressor(random_state=27))])

# Random Forest Pipeline
pipeline_randomforest=Pipeline([('rf_regressor',RandomForestRegressor(random_state=27))])

# XGBost Pipeline
pipeline_xgb=Pipeline([('xgb_regressor',xgb.XGBRegressor(objective="reg:linear", random_state=27))])

In [12]:
# Creating a list of the pipelines to loop through them
pipelines = [pipeline_dtr, pipeline_xgb, pipeline_randomforest]

best_accuracy=0.0
best_regressor=0
best_pipeline=""

# Dictionary of pipelines and regression types for ease of reference
pipe_dict = {0: 'DTR', 1: 'XGBoost', 2: 'RandomForest'}

# Fit the pipelines
for pipe in pipelines:
	pipe.fit(X_train, y_train)

from sklearn.metrics import accuracy_score
# Checking the accuracy of each model
for i,model in enumerate(pipelines):
    print("{} Test Accuracy: {}".format(pipe_dict[i],model.score(X_valid,y_valid)))

# Finding the best model
for i,model in enumerate(pipelines):
    if model.score(X_valid,y_valid)>best_accuracy:
        best_accuracy=model.score(X_valid,y_valid)
        best_pipeline=model
        best_regressor=i
print('Classifier with best accuracy: {}'.format(pipe_dict[best_regressor]))

[23:18:40] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[23:18:51] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
DTR Test Accuracy: 0.9728403633364292
XGBoost Test Accuracy: 0.9822455836271772
RandomForest Test Accuracy: 0.9845200900515633
Classifier with best accuracy: RandomForest


# Hyperparameters

### Random Forest Regressor

In [13]:
from sklearn.model_selection import RandomizedSearchCV
from pprint import pprint

In [14]:
# Using Randomized Search CV to find the best parameters

# Number of trees in random forest
n_estimators = [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 25, 50, 75, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]
# Method of selecting samples for training each tree
# bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

pprint(random_grid)

{'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 5, 10],
 'min_samples_split': [2, 5, 10, 15, 25, 50, 75, 100],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [15]:
# THIS BLOCK TAKES EONS
rf = RandomForestRegressor(random_state=27)
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=2, random_state=27, n_jobs = -1)

# Fit the random search model

rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 28.3min finished


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(random_state=27),
                   n_jobs=-1,
                   param_distributions={'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 5, 10],
                                        'min_samples_split': [2, 5, 10, 15, 25,
                                                              50, 75, 100],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=27, verbose=2)

In [16]:
# Evaluation of Random Search
def evaluate(model, X_valid, y_valid):
    predictions = model.predict(X_valid)
    errors = np.sqrt(mean_squared_error(y_valid, predictions))
    print('Model Performance')
    print('MSE of: ', errors)
    
    return errors

In [17]:
from sklearn.metrics import mean_squared_error
base_model = rf
base_model.fit(X_train, y_train)
base_accuracy = evaluate(base_model, X_valid, y_valid)


best_random = rf_random.best_estimator_
best_random.fit(X_train , y_train)

random_accuracy = evaluate(best_random, X_valid, y_valid)

print('\n')
print('Base Accuracy: ', base_accuracy)
print('\n')
print('Random Accuracy: ', random_accuracy)
print('Improvement of {:0.2f}%.'.format((random_accuracy - base_accuracy) / base_accuracy))

print('\n')
print('RF_Randomized_Search_CV is complete.')
print('\n')

Model Performance
MSE of:  32.063612748765806
Model Performance
MSE of:  31.58548726338043


Base Accuracy:  32.063612748765806


Random Accuracy:  31.58548726338043
Improvement of -0.01%.


RF_Randomized_Search_CV is complete.




In [18]:
print('The best model is',rf_random.best_estimator_)

The best model is RandomForestRegressor(max_depth=90, min_samples_split=5, n_estimators=2000,
                      random_state=27)


In [19]:
print('The accuracy of this model is', rf_random.best_score_*100)

The accuracy of this model is  98.12574395706247


Now to evaluate the accuracy of this hyperparametered model.

In [20]:
from sklearn.model_selection import cross_val_score

# Multiply by -1 since sklearn calculates *negative* MAE
scores = -1 * cross_val_score(best_random, X_valid, y_valid,
                              cv=5,
                              scoring='neg_mean_absolute_error')

print("MAE scores:\n", scores)
print("Average MAE score (across experiments):\n")
print(scores.mean())

KeyboardInterrupt: 

In [22]:
# Saving model to disk
import pickle
pickle.dump(best_random, open('model.pkl','wb'))

# Loading model to compare the results
model = pickle.load(open('model.pkl','rb'))

# Loading model to compare the results
model = pickle.load(open('model.pkl','rb'))